In [1]:
import torchvision.models as models

# Load the pre-trained SqueezeNet model
model = models.squeezenet1_1(pretrained=True)

In [2]:
print(model)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [2]:
import torch
from torchvision import transforms
from torchvision.models import squeezenet1_1
from PIL import Image
import datetime
import time
import pytz

def read_classes():
    """
    Load the ImageNet class names.
    """
    with open("imagenet-classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    return categories

print("Before loading model: ",datetime.datetime.now(pytz.timezone('Europe/London')))
# Load the pre-trained SqueezeNet model
model = squeezenet1_1(pretrained=True)
model.eval()  # Set the model to evaluation mode
categories = read_classes()

Before loading model:  2024-10-18 17:15:58.497908+01:00


In [21]:
# Load and preprocess an example image
print("Loading image: ",datetime.datetime.now(pytz.timezone('Europe/London')))
image_path = 'dog_1.jpg'
input_image = Image.open(image_path).convert('RGB')

Loading image:  2024-10-18 17:30:47.709394+01:00


In [22]:
print("Image transformation: ",datetime.datetime.now(pytz.timezone('Europe/London')))
# Define the image transformation pipeline
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply transformations to the input image
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension


Image transformation:  2024-10-18 17:30:49.373018+01:00


In [23]:
# Make the prediction
with torch.no_grad():
    start_time = time.time()
    print("During prediction: ", datetime.datetime.now(pytz.timezone('Europe/London')))
    # print("During prediction: ",jtop().stats)
    output = model(input_batch)
    end_time = time.time()

# Get the predicted class index
# get the softmax probabilities
probabilities = torch.nn.functional.softmax(output[0], dim=0)
# check the top category that are predicted
top5_prob, top5_catid = torch.topk(probabilities, 1)

# Print the predicted class label and inference time
inference_time = end_time - start_time
print(f'Inference time: {inference_time:.4f} seconds')
print(f"Accuracy: {top5_prob[0].item()*100:.3f}%")
print(f"Predicted class:{categories[top5_catid[0]]}")      

During prediction:  2024-10-18 17:30:51.788812+01:00
Inference time: 0.2037 seconds
Accuracy: 28.500%
Predicted class:Newfoundland, Newfoundland dog


In [7]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total trainable parameters: {total_params}')

Total trainable parameters: 1235496


In [8]:
import torch
import torchvision.models as models

# Load the pre-trained SqueezeNet model
model = models.squeezenet1_1(pretrained=True)

# Define input size (224x224x3) for SqueezeNet
input_size = (224, 224, 3)

# Function to calculate FLOPs for the model
def count_flops(model, input_size):
    # Define a tensor of appropriate size
    input_tensor = torch.randn(1, *input_size)
    
    # Switch to evaluation mode
    model.eval()
    
    # Move model to appropriate device
    device = next(model.parameters()).device
    input_tensor = input_tensor.to(device)
    model.to(device)
    
    # Iterate through model's layers
    flops = 0
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            # For convolutional layers
            output_size = (input_size[0] - module.kernel_size[0] + 2 * module.padding[0]) // module.stride[0] + 1
            output_size = (output_size - module.kernel_size[1] + 2 * module.padding[1]) // module.stride[1] + 1
            flops += module.in_channels * module.out_channels * module.kernel_size[0] * module.kernel_size[1] * output_size * output_size
            input_size = (output_size, output_size, module.out_channels)
        elif isinstance(module, torch.nn.MaxPool2d):
            # For max pooling layers
            output_size = (input_size[0] - module.kernel_size) // module.stride + 1
            flops += input_size[2] * output_size * output_size
            input_size = (output_size, output_size, input_size[2])
        elif isinstance(module, torch.nn.Linear):
            # For fully connected layers
            flops += module.in_features * module.out_features
            input_size = (module.out_features,)
    
    return flops

# Calculate FLOPs
total_flops = count_flops(model, input_size)
print("Total FLOPs:", total_flops)

Total FLOPs: 78414144


Total FLOPs: 78414144
Before loading model:  2024-10-18 17:15:58.497908+01:00

0: 1.jpg

Loading image:  2024-10-18 17:16:03.225230+01:00
Image transformation:  2024-10-18 17:16:06.776816+01:00
During prediction:  2024-10-18 17:16:47.169217+01:00
Inference time: 0.2406 seconds
Accuracy: 36.797%
Predicted class:Italian greyhound

datetime.datetime(2024, 10, 18, 17, 15, 58, 473191)
datetime.datetime(2024, 10, 18, 17, 15, 59, 474734)
datetime.datetime(2024, 10, 18, 17, 16, 00, 471542)
datetime.datetime(2024, 10, 18, 17, 16, 01, 487971)
datetime.datetime(2024, 10, 18, 17, 16, 02, 483455)
datetime.datetime(2024, 10, 18, 17, 16, 03, 477773)
datetime.datetime(2024, 10, 18, 17, 16, 04, 488956)
datetime.datetime(2024, 10, 18, 17, 16, 05, 491392)
datetime.datetime(2024, 10, 18, 17, 16, 06, 496376)
datetime.datetime(2024, 10, 18, 17, 16, 07, 488938)
datetime.datetime(2024, 10, 18, 17, 16, 08, 495191)
datetime.datetime(2024, 10, 18, 17, 16, 09, 487683)
datetime.datetime(2024, 10, 18, 17, 16, 10, 492668)
datetime.datetime(2024, 10, 18, 17, 16, 11, 531887)
datetime.datetime(2024, 10, 18, 17, 16, 12, 498758)
datetime.datetime(2024, 10, 18, 17, 16, 13, 503234)
datetime.datetime(2024, 10, 18, 17, 16, 14, 505264)
datetime.datetime(2024, 10, 18, 17, 16, 15, 508592)
datetime.datetime(2024, 10, 18, 17, 16, 16, 495212)
datetime.datetime(2024, 10, 18, 17, 16, 17, 510283)
datetime.datetime(2024, 10, 18, 17, 16, 18, 515166)
datetime.datetime(2024, 10, 18, 17, 16, 19, 510467)
datetime.datetime(2024, 10, 18, 17, 16, 20, 514992)
datetime.datetime(2024, 10, 18, 17, 16, 21, 518498)
datetime.datetime(2024, 10, 18, 17, 16, 22, 504932)
datetime.datetime(2024, 10, 18, 17, 16, 23, 515739)
datetime.datetime(2024, 10, 18, 17, 16, 24, 513081)
datetime.datetime(2024, 10, 18, 17, 16, 25, 508490)
datetime.datetime(2024, 10, 18, 17, 16, 26, 520334)
datetime.datetime(2024, 10, 18, 17, 16, 27, 522590)
datetime.datetime(2024, 10, 18, 17, 16, 28, 527114)
datetime.datetime(2024, 10, 18, 17, 16, 29, 521484)
datetime.datetime(2024, 10, 18, 17, 16, 30, 514442)
datetime.datetime(2024, 10, 18, 17, 16, 31, 523594)
datetime.datetime(2024, 10, 18, 17, 16, 32, 526074)
datetime.datetime(2024, 10, 18, 17, 16, 33, 519163)
datetime.datetime(2024, 10, 18, 17, 16, 34, 529945)
datetime.datetime(2024, 10, 18, 17, 16, 35, 520158)
datetime.datetime(2024, 10, 18, 17, 16, 36, 532454)
datetime.datetime(2024, 10, 18, 17, 16, 37, 532545)
datetime.datetime(2024, 10, 18, 17, 16, 38, 535359)
datetime.datetime(2024, 10, 18, 17, 16, 39, 538522)
datetime.datetime(2024, 10, 18, 17, 16, 40, 536767)
datetime.datetime(2024, 10, 18, 17, 16, 41, 544925)
datetime.datetime(2024, 10, 18, 17, 16, 42, 539870)
datetime.datetime(2024, 10, 18, 17, 16, 43, 545569)
datetime.datetime(2024, 10, 18, 17, 16, 44, 539006)
datetime.datetime(2024, 10, 18, 17, 16, 45, 535824)
datetime.datetime(2024, 10, 18, 17, 16, 46, 546790)
datetime.datetime(2024, 10, 18, 17, 16, 47, 540044)

Average CPU Temperature: 30.56°C
Average GPU Temperature: 31.67°C
Average Power CPU: 607.02mW
Average Power GPU: 607.02mW
Average Power TOT: 2003.18mW
Average RAM: 0.4492

{'Avg_CPU_temp': 30.56, 'Avg_GPU_temp': 31.67, 'Avg_Power_CPU': 607.02, 'Avg_Power_GPU': 607.02, 'Avg_CPU_RAM': 0.4491608675515615}

1: cat_0

Loading image:  2024-10-18 17:20:34.085449+01:00
Image transformation:  2024-10-18 17:20:37.033955+01:00
During prediction:  2024-10-18 17:20:40.340777+01:00
Inference time: 0.1844 seconds
Accuracy: 21.240%
Predicted class:tabby, tabby cat

datetime.datetime(2024, 10, 18, 17, 20, 34, 852576)
datetime.datetime(2024, 10, 18, 17, 20, 35, 863440)
datetime.datetime(2024, 10, 18, 17, 20, 36, 865007)
datetime.datetime(2024, 10, 18, 17, 20, 37, 865598)
datetime.datetime(2024, 10, 18, 17, 20, 38, 870484)
datetime.datetime(2024, 10, 18, 17, 20, 39, 873867)
datetime.datetime(2024, 10, 18, 17, 20, 40, 851437)

Average CPU Temperature: 30.93°C
Average GPU Temperature: 31.57°C
Average Power CPU: 646.43mW
Average Power GPU: 646.43mW
Average Power TOT: 1989.29mW
Average RAM: 0.4511

{'Avg_CPU_temp': 30.928571428571427, 'Avg_GPU_temp': 31.571428571428573, 'Avg_Power_CPU': 646.4285714285714, 'Avg_Power_GPU': 646.4285714285714, 'Avg_CPU_RAM': 0.4510510764154304}

2: cat_1

Loading image:  2024-10-18 17:23:27.504687+01:00
Image transformation:  2024-10-18 17:23:29.394820+01:00
During prediction:  2024-10-18 17:23:32.067847+01:00
Inference time: 0.2234 seconds
Accuracy: 50.415%
Predicted class:tabby, tabby cat

datetime.datetime(2024, 10, 18, 17, 23, 27, 098893)
datetime.datetime(2024, 10, 18, 17, 23, 28, 099090)
datetime.datetime(2024, 10, 18, 17, 23, 29, 103469)
datetime.datetime(2024, 10, 18, 17, 23, 30, 100937)
datetime.datetime(2024, 10, 18, 17, 23, 31, 101939)
datetime.datetime(2024, 10, 18, 17, 23, 32, 130194)

Average CPU Temperature: 30.50°C
Average GPU Temperature: 31.83°C
Average Power CPU: 652.33mW
Average Power GPU: 652.33mW
Average Power TOT: 2067.67mW
Average RAM: 0.4508

{'Avg_CPU_temp': 30.5, 'Avg_GPU_temp': 31.833333333333332, 'Avg_Power_CPU': 652.3333333333334, 'Avg_Power_GPU': 652.3333333333334, 'Avg_CPU_RAM': 0.4508495843228453}

3: dog_0

Loading image:  2024-10-18 17:25:17.763404+01:00
Image transformation:  2024-10-18 17:25:19.575718+01:00
During prediction:  2024-10-18 17:25:21.917458+01:00
Inference time: 0.1840 seconds
Accuracy: 60.378%
Predicted class:malamute, malemute, Alaskan malamute

datetime.datetime(2024, 10, 18, 17, 25, 17, 269413)
datetime.datetime(2024, 10, 18, 17, 25, 18, 266322)
datetime.datetime(2024, 10, 18, 17, 25, 19, 255503)
datetime.datetime(2024, 10, 18, 17, 25, 20, 263574)
datetime.datetime(2024, 10, 18, 17, 25, 21, 260694)
datetime.datetime(2024, 10, 18, 17, 25, 22, 256494)

Average CPU Temperature: 30.92°C
Average GPU Temperature: 31.83°C
Average Power CPU: 838.50mW
Average Power GPU: 838.50mW
Average Power TOT: 2199.50mW
Average RAM: 0.4509

{'Avg_CPU_temp': 30.916666666666668, 'Avg_GPU_temp': 31.833333333333332, 'Avg_Power_CPU': 838.5, 'Avg_Power_GPU': 838.5, 'Avg_CPU_RAM': 0.450908380222242}

4: cat_2

Loading image:  2024-10-18 17:26:44.837699+01:00
Image transformation:  2024-10-18 17:26:47.417476+01:00
During prediction:  2024-10-18 17:26:50.994242+01:00
Inference time: 0.1520 seconds
Accuracy: 99.602%
Predicted class:Persian cat

datetime.datetime(2024, 10, 18, 17, 26, 44, 383309)
datetime.datetime(2024, 10, 18, 17, 26, 45, 383375)
datetime.datetime(2024, 10, 18, 17, 26, 46, 377124)
datetime.datetime(2024, 10, 18, 17, 26, 47, 383045)
datetime.datetime(2024, 10, 18, 17, 26, 48, 385981)
datetime.datetime(2024, 10, 18, 17, 26, 49, 387068)
datetime.datetime(2024, 10, 18, 17, 26, 50, 389945)
datetime.datetime(2024, 10, 18, 17, 26, 51, 386007)

Average CPU Temperature: 30.88°C
Average GPU Temperature: 31.88°C
Average Power CPU: 746.88mW
Average Power GPU: 746.88mW
Average Power TOT: 2114.88mW
Average RAM: 0.4510

{'Avg_CPU_temp': 30.875, 'Avg_GPU_temp': 31.875, 'Avg_Power_CPU': 746.875, 'Avg_Power_GPU': 746.875, 'Avg_CPU_RAM': 0.4510115193977198}

5: dog_1

Loading image:  2024-10-18 17:30:47.709394+01:00
Image transformation:  2024-10-18 17:30:49.373018+01:00
During prediction:  2024-10-18 17:30:51.788812+01:00
Inference time: 0.2037 seconds
Accuracy: 28.500%
Predicted class:Newfoundland, Newfoundland dog

datetime.datetime(2024, 10, 18, 17, 30, 47, 731809)
datetime.datetime(2024, 10, 18, 17, 30, 48, 725311)
datetime.datetime(2024, 10, 18, 17, 30, 49, 720824)
datetime.datetime(2024, 10, 18, 17, 30, 50, 726526)
datetime.datetime(2024, 10, 18, 17, 30, 51, 719657)
datetime.datetime(2024, 10, 18, 17, 30, 52, 727802)

Average CPU Temperature: 30.58°C
Average GPU Temperature: 31.75°C
Average Power CPU: 503.67mW
Average Power GPU: 503.67mW
Average Power TOT: 1887.17mW
Average RAM: 0.4509

{'Avg_CPU_temp': 30.583333333333332, 'Avg_GPU_temp': 31.75, 'Avg_Power_CPU': 503.6666666666667, 'Avg_Power_GPU': 503.6666666666667, 'Avg_CPU_RAM': 0.4508814457878814}